In [1]:
import os
import pandas as pd
import numpy as np
import sys
import re
import logging
from Modules.Loader_wrangler import *
import random
import torch
import torch.nn as nn
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler

In [24]:
# Configure basic logging
logging.basicConfig(level=logging.INFO, force=True, format='%(levelname)s: %(message)s')

In [ ]:
play = loader(output_file_name="merged_df2017.pkl", chunksize=100000, sample_size=100000, survey_year=2017)

In [6]:
play = pd.read_pickle("/home/trapfishscott/Cambridge24.25/D200_ML_econ/ProblemSets/Project/data/merged_df2017.pkl")

### Obtaining only relevant variables and making into a time series

In [9]:
ts_df = ts_df.sort_values(["IndividualID_x", "TravelWeekDay_B01ID", "JourSeq"])

## Data Manipulation pipeline

1. One-hot encode categorical features + any small cleaning steps
2. Add days of the week with no car travel
3. Make data frame into wide format
4. Convert to tensor

* Includes JourSeq gaps if trips were made by non-car inbetween

In [ ]:
### small cleaning steps and one hot encoding

In [ ]:
#temporal_vars = ["TWSMonth", "TravelYear", "TravelWeekDay_B01ID"]
#individual_vars =["PSUGOR_B02ID", "IndIncome2002_B02ID", "HHoldNumChildren", "DVLALengthBand_B01ID"]

numerical_outcome_vars = ["TripStart", "TripEnd", "TripDisExSW"]
categorical_outcome_vars = ["TripPurpose_B01ID"]


extra_vars = ["IndividualID_x", "JourSeq"]


features_one_hot = ["PSUGOR_B02ID"]
features_numerical = ["TravelYear", "HHoldNumChildren", "IndIncome2002_B02ID", "DVLALengthBand_B01ID"]
features_cyclical = ["TWSMonth", "TravelWeekDay_B01ID"]

features = features_one_hot + features_numerical + features_cyclical
outcomes = numerical_outcome_vars + categorical_outcome_vars

In [33]:
ts_df = play[extra_vars +  features + outcomes]

### Encoders

In [ ]:
# Cyclical encoder

def apply_cyclical_encoding(column, type_, max_val):

    if type_ == "cos":
        return np.cos(2 * np.pi * column/ max_val)
    else:
        return np.sin(2 * np.pi * column/ max_val)


mms = MinMaxScaler()


### Minute-from-midnight encoder

### Imputing missing travel days

In [26]:
def impute_missing_travel_week_for_i(i_df, i_id, full_week_encoding, features=features, outcomes=outcomes):
        
    break_flag = False

    # Travel days with travel 
    included_travel_day = i_df["TravelWeekDay_B01ID"].to_list()

    # Travel days with no travel
    travel_day_no_drive = list(set(full_week_encoding) - set(included_travel_day))

    # These values will repeat for empty-travel travel days
    imputed_travel_df = pd.DataFrame({
        "TravelWeekDay_B01ID": travel_day_no_drive,
        "IndividualID_x": [i_id]*len(travel_day_no_drive),
        "JourSeq": [1]*len(travel_day_no_drive)
    })

    # Looping through all the columns in the original df
    for col in i_df.columns:

        # For days with no travel all outcomes vars will take 0
        if col in outcomes:
            imputed_travel_df[col] = [0]*len(travel_day_no_drive)

        else:
        
            if col not in extra_vars + ["TravelWeekDay_B01ID"]:
                if len(i_df[col].unique()) != 1:
                    print(f"{col} is erroneous for {i_id}")
                    print(f"Unique vals: {i_df[col].unique()}")
                    break_flag = True
                    break
                else:
                    imputed_travel_df[col] = i_df[col].unique()[0]

    if break_flag:
        print("Continuing to next individual")
        return
    

    # display(imputed_travel_df)

    # Concatenating df to include empty travel days
    full_df = pd.concat([i_df, imputed_travel_df])

    # Apply cyclical encoding to cyclical column

    full_df["TWSMonth_cos"] = apply_cyclical_encoding(column=full_df["TWSMonth"], type_="cos", max_val=12)
    full_df["TWSMonth_sin"] =apply_cyclical_encoding(column=full_df["TWSMonth"], type_="sin", max_val=12)

    full_df["TravelWeekDay_B01ID_cos"] = apply_cyclical_encoding(column=full_df["TravelWeekDay_B01ID"], type_="cos", max_val=7)
    full_df["TravelWeekDay_B01ID_sin"] =apply_cyclical_encoding(column=full_df["TravelWeekDay_B01ID"], type_="sin", max_val=7)

    full_df = full_df.sort_values(["TravelYear", "TWSMonth", "TravelWeekDay_B01ID", "JourSeq", "TripStart", "TripEnd"])

    #display(full_df)


    return full_df

### Transforming to wide

In [27]:
def transform_to_wide_for_i(i_df, max_journey_seq, seq_length = 7, outcomes=outcomes, features=features, extra_vars=extra_vars):
    df = i_df.copy()

    expected_all = [f"{col}_{i}" for col in outcomes for i in range(1, max_journey_seq+1)]
    expected_categorical = [f"{col}_{i}" for col in categorical_outcome_vars for i in range(1, max_journey_seq+1)]

    df = df[df["JourSeq"]<=max_journey_seq]

    #

    df_wide = df.pivot(index="TravelWeekDay_B01ID",
                  columns = "JourSeq",
                  values = outcomes)
    
    df_wide.columns = [f"{col[0]}_{int(col[1])}" for col in df_wide.columns]

    for col in expected_all:
        if col not in df_wide.columns:
            df_wide[col] = 0
    
    # Ensure column order is consistent
    df_wide = df_wide[expected_all]
    
    df_wide = df_wide.fillna(0)

    df_wide.reset_index(inplace=True)

    # Dropping outcome columns
    df.drop(columns=outcomes + extra_vars, axis=1, inplace = True)
    df.drop_duplicates(subset=["TravelWeekDay_B01ID"], inplace=True)

    df_wide = df_wide.merge(df, on="TravelWeekDay_B01ID", how="left")

    top_row = df_wide.head(1).copy()

    for col in expected_all:
        top_row[col] = 0
        top_row["TravelWeekDay_B01ID"] = 0

    repeated_rows = pd.concat([top_row] * seq_length, ignore_index=True)

    df_wide = pd.concat([repeated_rows, df_wide], ignore_index=True)

    df_wide.drop(columns=features_one_hot + features_cyclical, inplace=True, axis=1)

    #df_wide.drop(columns=features_cyclical + features_one_hot, axis=1, inplace=True)

    targets_only = df_wide.drop(columns=features + extra_vars, axis=1, errors="ignore")

    targets_only = targets_only.iloc[seq_length:,:]

    targets_cont = targets_only[expected_all]
    targets_cont = targets_cont.copy()
    targets_cont.drop(columns=expected_categorical, axis=1, inplace=True)


    targets_cat = targets_only[expected_categorical]

    return df_wide, targets_cont, targets_cat

### Putting altogether for LSTM

In [ ]:
def prepare_data_for_LSTM(long_df, impute_missing_travel_weeks=True, transform_to_wide=False, transform_to_tensor=False, debug=False):

    df = long_df.copy()
           

    #df = df[~df["DVLALengthBand_B01ID"].isin([-8, -10])]

    # All unique individual id's to loop over
    individual_ids = df["IndividualID_x"].unique()

    # Apply numerical encoding to numerical column
    #num_scaler = MinMaxScaler()

    df[numerical_outcome_vars] = num_scaler.fit_transform(df[numerical_outcome_vars])
    df[features_numerical] = num_scaler.fit_transform(df[features_numerical])

    for col in features_one_hot:
        df[col] = df[col].astype(int)
    
    # Apply one-hot to categorical
    ohe = OneHotEncoder(sparse_output=False)

    ohe_array = ohe.fit_transform(df[features_one_hot])
    ohe_df = pd.DataFrame(ohe_array, columns=ohe.get_feature_names_out(features_one_hot))

    # Reset index to avoid misalignment
    df.reset_index(drop=True, inplace=True)
    ohe_df.reset_index(drop=True, inplace=True)

    df = pd.concat([df, ohe_df], axis=1)

    df_chunks = []

    full_week_encoding = list(range(1,8))

    if debug:
        random_index = random.randint(0, len(individual_ids))

        debug_df = df[df["IndividualID_x"] == individual_ids[random_index]]

        #display(debug_df)

        debug_df = impute_missing_travel_week_for_i(debug_df, i_id=individual_ids[random_index], full_week_encoding=full_week_encoding)

        #display(debug_df)

        debug_df, debug_targets_cont, debug_targets_cat = transform_to_wide_for_i(debug_df, max_journey_seq=10)

        #for i, col in enumerate(debug_targets_cat.columns):
        #    print(f"{i}: {col}")

        #print(debug_df.iloc[:,[0,1,2,3,20,31]].to_latex())

        #display(debug_df.iloc[0:7,[0,1,2,46]])

        display(debug_df)

        display(debug_targets_cont)

        display(debug_targets_cat)

        return
    
    if transform_to_tensor:
        individual_tensors = []
        target_cont_tensors = []
        target_cat_tensors = []
    
    if impute_missing_travel_weeks:

        for i, individual_id in enumerate(individual_ids[:]):

            i_df = df[df["IndividualID_x"] == individual_id]

            full_df = impute_missing_travel_week_for_i(i_df, i_id=individual_id, full_week_encoding=full_week_encoding)

            #display(full_df)

            if full_df is not None:
                if not transform_to_wide:
                    df_chunks.append(full_df)

                else:

                    full_df, targets_cont, targets_cat = transform_to_wide_for_i(full_df, max_journey_seq=10)
                    
                    if transform_to_tensor:

                    
                        full_arr = full_df.to_numpy()
                        full_arr = np.expand_dims(full_arr, axis=1)

                        targets_cont_arr = targets_cont.to_numpy()
                        targets_cat_arr = targets_cat.to_numpy()

                        full_i_tensor = torch.tensor(full_arr)
                        target_cont_i_tensor = torch.tensor(targets_cont_arr)
                        target_cat_i_tensor = torch.tensor(targets_cat_arr)

                        individual_tensors.append(full_i_tensor)
                        target_cont_tensors.append(target_cont_i_tensor)
                        target_cat_tensors.append(target_cat_i_tensor)


                    else:

                        #display(full_df)
                        print("")
                        #display(targets)
                        df_chunks.append(full_df)

            sys.stdout.write(f"\rIndividual {i+1} out of {len(individual_ids)} Complete!    ")
            sys.stdout.flush()

        if transform_to_tensor:
            individual_tensors = torch.stack(individual_tensors, dim=0)
            target_cont_tensors = torch.stack(target_cont_tensors, dim=0)
            target_cat_tensors = torch.stack(target_cat_tensors, dim=0)
            return individual_tensors, target_cont_tensors, target_cat_tensors
        
        else:

            df_to_return = pd.concat(df_chunks)

            return df_to_return



    else:
        return df


In [29]:
df = prepare_data_for_LSTM(long_df=ts_df, debug=True)


,TripStart_1,TripStart_2,TripStart_3,TripStart_4,TripStart_5,TripStart_6,TripStart_7,TripStart_8,TripStart_9,TripStart_10,...,PSUGOR_B02ID_4,PSUGOR_B02ID_5,PSUGOR_B02ID_6,PSUGOR_B02ID_7,PSUGOR_B02ID_8,PSUGOR_B02ID_9,TWSMonth_cos,TWSMonth_sin,TravelWeekDay_B01ID_cos,TravelWeekDay_B01ID_sin
0,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,6.123234e-17,1.0,0.623490,7.818315e-01
1,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,6.123234e-17,1.0,0.623490,7.818315e-01
2,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,6.123234e-17,1.0,0.623490,7.818315e-01
3,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,6.123234e-17,1.0,0.623490,7.818315e-01
4,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,6.123234e-17,1.0,0.623490,7.818315e-01
5,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,6.123234e-17,1.0,0.623490,7.818315e-01
6,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,6.123234e-17,1.0,0.623490,7.818315e-01
7,0.434329,0.524670,0.000000,0.000000,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,6.123234e-17,1.0,0.623490,7.818315e-01
8,0.441279,0.538568,0.781793,0.899931,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,6.123234e-17,1.0,-0.222521,9.749279e-01
9,0.479500,0.514246,0.597637,0.618485,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,6.123234e-17,1.0,-0.900969,4.338837e-01


,TripStart_1,TripStart_2,TripStart_3,TripStart_4,TripStart_5,TripStart_6,TripStart_7,TripStart_8,TripStart_9,TripStart_10,...,TripDisExSW_1,TripDisExSW_2,TripDisExSW_3,TripDisExSW_4,TripDisExSW_5,TripDisExSW_6,TripDisExSW_7,TripDisExSW_8,TripDisExSW_9,TripDisExSW_10
7,0.434329,0.524670,0.000000,0.000000,0,0,0,0,0,0,...,0.046929,0.046929,0.000000,0.000000,0,0,0,0,0,0
8,0.441279,0.538568,0.781793,0.899931,0,0,0,0,0,0,...,0.153832,0.002537,0.003805,0.177387,0,0,0,0,0,0
9,0.479500,0.514246,0.597637,0.618485,0,0,0,0,0,0,...,0.014314,0.014314,0.001812,0.001812,0,0,0,0,0,0
10,0.712300,0.719249,0.795691,0.813065,0,0,0,0,0,0,...,0.001631,0.001631,0.001812,0.001812,0,0,0,0,0,0
11,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0
12,0.396108,0.896456,0.000000,0.000000,0,0,0,0,0,0,...,0.113970,0.113970,0.000000,0.000000,0,0,0,0,0,0
13,0.639333,0.653231,0.802641,0.854760,0,0,0,0,0,0,...,0.001812,0.001812,0.014314,0.014314,0,0,0,0,0,0


,TripPurpose_B01ID_1,TripPurpose_B01ID_2,TripPurpose_B01ID_3,TripPurpose_B01ID_4,TripPurpose_B01ID_5,TripPurpose_B01ID_6,TripPurpose_B01ID_7,TripPurpose_B01ID_8,TripPurpose_B01ID_9,TripPurpose_B01ID_10
7,9.0,9.0,0.0,0.0,0,0,0,0,0,0
8,5.0,10.0,6.0,6.0,0,0,0,0,0,0
9,23.0,23.0,6.0,6.0,0,0,0,0,0,0
10,9.0,9.0,5.0,5.0,0,0,0,0,0,0
11,0.0,0.0,0.0,0.0,0,0,0,0,0,0
12,6.0,6.0,0.0,0.0,0,0,0,0,0,0
13,5.0,5.0,23.0,23.0,0,0,0,0,0,0


In [134]:
X, y_cont, y_cat = prepare_data_for_LSTM(long_df=ts_df, transform_to_wide=True, transform_to_tensor=True)

Individual 5949 out of 6838 Complete!    TravelYear is erroneous for 2017014397.0
Unique vals: [1. 0.]
Continuing to next individual
Individual 5950 out of 6838 Complete!    TravelYear is erroneous for 2017014398.0
Unique vals: [1. 0.]
Continuing to next individual
Individual 5998 out of 6838 Complete!    TravelYear is erroneous for 2017014552.0
Unique vals: [1. 0.]
Continuing to next individual
Individual 6057 out of 6838 Complete!    TravelYear is erroneous for 2017014714.0
Unique vals: [1. 0.]
Continuing to next individual
Individual 6058 out of 6838 Complete!    TravelYear is erroneous for 2017014715.0
Unique vals: [1. 0.]
Continuing to next individual
Individual 6086 out of 6838 Complete!    TravelYear is erroneous for 2017014773.0
Unique vals: [1. 0.]
Continuing to next individual
Individual 6164 out of 6838 Complete!    TravelYear is erroneous for 2017014964.0
Unique vals: [1. 0.]
Continuing to next individual
Individual 6165 out of 6838 Complete!    TravelYear is erroneous for 

In [137]:
# Save tensors
with open("/home/trapfishscott/Cambridge24.25/D200_ML_econ/ProblemSets/Project/tensors/tensors.pkl", "wb") as f:
    pickle.dump((X, y_cont, y_cat), f)

In [4]:
#Load tensors
with open("/home/trapfishscott/Cambridge24.25/D200_ML_econ/ProblemSets/Project/tensors/tensors.pkl", "rb") as f:
    (X, y_cont, y_cat) = pickle.load(f)

In [16]:
X = X.to(torch.float32)
y_cont = y_cont.to(torch.float32)
y_cat = y_cat.to(torch.long)

print(f"Input shape: {X.shape}")
print(f"Cont Output shape: {y_cont.shape}")
print(f"Cat Output shape: {y_cat.shape}")

Input shape: torch.Size([6775, 14, 1, 57])
Cont Output shape: torch.Size([6775, 7, 30])
Cat Output shape: torch.Size([6775, 7, 10])


### Creating the RNN

In [17]:
# Defining parameters
INPUT_SIZE = X.shape[3]
HIDDEN_SIZE = 3
NUM_LAYERS = 1
OUTPUT_SIZE_CONT = y_cont.shape[2]
OUTPUT_SIZE_CAT = y_cat.shape[2]

NUM_CLASSES = int(ts_df["TripPurpose_B01ID"].max())+1
print(NUM_CLASSES)

24


In [18]:
class RNNmodel(nn.Module):
    def __init__(self):
        super().__init__()

        # Define RNN layer

        self.rnn = nn.RNN(INPUT_SIZE, HIDDEN_SIZE)

        # Output layers

        self.output_cont = nn.Linear(HIDDEN_SIZE, OUTPUT_SIZE_CONT)
        self.output_cat = nn.Linear(HIDDEN_SIZE, OUTPUT_SIZE_CAT)


    def forward(self, X):

        out, hh = self.rnn(X)

        y_cont_hat_vector = self.output_cont(hh)

        y_cat_hat= self.output_cat(hh)

        y_cat_hat = y_cat_hat.permute(0,2,1)
        y_cat_hat = torch.cat([y_cat_hat]*NUM_CLASSES, dim=2)
        y_cat_hat = y_cat_hat.reshape(OUTPUT_SIZE_CAT, NUM_CLASSES)

        # stacking downward NUM_CLASSES times
        #y_cat_hat = y_cat_hat.repeat()
        #print(y_cat_hat)

        y_cont_hat = y_cont_hat_vector[0,0,:]

        y_cont_hat = y_cont_hat.to(torch.float32)
        y_cat_hat = y_cat_hat.to(torch.float32)

        # appplying relu so that continous values are non-negative
        y_cont_hat = torch.relu(y_cont_hat)

        return y_cont_hat, y_cat_hat


In [30]:
# Taking one test draw

rnn_model = RNNmodel()

X0 = X[0,:,0,:].unsqueeze(1).to(torch.float32)
print(f"X1 shape: {X0.shape}")
print("")

y_cont_hat, y_cat_hat = rnn_model.forward(X0)

y_cat_i = y_cat[0,0,:]

print(f"Categorical outputs:  {y_cat_hat.shape}")
print(f"Ground truth categorical: {y_cat_i.shape}")
print("")
print(f"Continous outputs:  {y_cont_hat}")
print(f"Ground truth Continous: {y_cont[0,0,:].shape}")

loss_cat = nn.CrossEntropyLoss()  #(y_hat, y)
loss_cont = nn.MSELoss()

print(f"Categorical loss: {loss_cat(y_cat_hat, y_cat_i)}")
print(f"Continous loss: {loss_cont(y_cont_hat, y_cont[0,0,:])}")


X1 shape: torch.Size([14, 1, 57])

Categorical outputs:  torch.Size([10, 24])
Ground truth categorical: torch.Size([10])

Continous outputs:  tensor([0.4656, 0.0000, 0.1246, 0.0000, 1.0922, 0.8097, 0.6332, 0.0000, 0.1434,
        0.0000, 0.2216, 0.0000, 0.0688, 0.0000, 0.1103, 0.0000, 0.0000, 0.0000,
        0.5133, 0.0000, 0.0000, 0.6024, 0.2155, 0.0000, 0.0000, 0.0000, 0.3909,
        0.3429, 0.0000, 0.0000], grad_fn=<ReluBackward0>)
Ground truth Continous: torch.Size([30])
Categorical loss: 3.178054094314575
Continous loss: 0.12522464990615845


In [23]:
rnn_model = RNNmodel()

loss_cat = nn.CrossEntropyLoss()  #(y_hat, y)
loss_cont = nn.MSELoss()
optimizer = torch.optim.Adam(rnn_model.parameters(), lr=0.001)

epochs = 1

seq_length = 7

for epochi in range(epochs):
    #print(epochi)
    for individual_i in range(300):

        travel_diary = X[individual_i, :, 0, :].unsqueeze(1).to(torch.float32)

        for i in range(1, travel_diary.shape[0] - seq_length):
            sliding_input = travel_diary[i:seq_length+i,0,:].unsqueeze(1)

            #print(sliding_input.shape)

            y_cont_hat, y_cat_hat_logit = rnn_model.forward(sliding_input)

            y_cont_i = y_cont[individual_i,i,:]
            y_cat_i = y_cat[individual_i,i,:].long()

            categorical_prediction = torch.argmax(y_cat_hat, dim=-1)


            #print(f"Truth: {y_cat_i}")
            #print(f"Prediction: {categorical_prediction}") 
            #print(y_cat_hat_logit)

            #print(f"Cont pred: {y_cont_hat}")
            #print(f"Cont true: {y_cont_i}")
            #print(y_cat_i.shape)
            #print(y_cat_hat_logit.shape)
            #print(y_cont_i.shape)
            #print(y_cont_hat.shape)
            #print("")


            categorical_loss = loss_cat(y_cat_hat_logit, y_cat_i)


            continuous_loss = loss_cont(y_cont_hat, y_cont_i)


            combined_loss = 0.7*categorical_loss + 0.3*continuous_loss
            
            
            optimizer.zero_grad()
            combined_loss.backward()
            optimizer.step()
            
            
            
            print(f"\repoch: {epochi} | individual: {individual_i} | categorical loss: {categorical_loss:.2f} | continuous loss: {continuous_loss:2f}", end="", flush=True)

            



    


epoch: 0 | individual: 299 | categorical loss: 3.18 | continuous loss: 0.007576